# Statistical Analysis of Bay Area Bike Share Data

> <b>1</b> Is Commuter ridership affected by Rain?
> 
> <b>2</b> Is Commuter ridership affected by Hot or Cold Temperatures?

In [1]:
%matplotlib inline

import matplotlib
import numpy as np
# from scipy import stats
# import scipy
import math
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob
import datetime

import seaborn as sns
# sns.set()
sns.set_style('whitegrid')
sns.set_context("poster")

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

font = {'size'   : 50}
matplotlib.rc('font', **font)

TITLE_FONT_SIZE = 25
LABEL_FONT_SIZE = 15
TICK_FONT_SIZE  = 15

day_labels = ['MON','TUE','WED','THU','FRI','SAT','SUN']
day_labels_full = ['MONDAY','TUESDAY','WEDNESDAY','THURSDAY','FRIDAY','SATURDAY','SUNDAY']
month_labels = ['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC']
month_labels_full = ['JANUARY','FEBRUARY','MARCH','APRIL','MAY','JUNE','JULY','AUGUST','SEPTEMBER','OCTOBER','NOVEMBER','DECEMBER']

sub_color = 'b'
sub_color_alt = 'm'
cust_color='r'
cust_color_alt='y'

commuter_color='g'
commuter_color_alt='#1daf1d'

commuter_am = '#ea54d9'     #OrRd
commuter_am_alt = '#9b8460' #OrRd_r

commuter_pm = '#b97ccc'     #PuRd
commuter_pm_alt = '#f4ad3a' #PuRd_r

FIG_SIZE = (15,6)
FIG_SIZE_SHORT = (15,3)
GRID_DIMS = 15

DO_WRITE_CHARTS = True

# Load Morning and Evening Commuter Trips Data

In [3]:
super_stations = [69, 70]

In [4]:
print('[%s] Loading Morning Commute Trips Data...' % datetime.datetime.now().time())

morning_commutes = pd.DataFrame()
trip_data_file = '../../clean_data/bayareabikeshare/trip_data_morning_commutes.csv'

# Chunk Settings
chunks = []
chunk_counter = 1
chunksize = 10000
num_chunks = math.ceil(sum(1 for row in open(trip_data_file, 'r'))/chunksize)

# import file in chunks
for chunk in pd.read_csv(trip_data_file, chunksize=chunksize, iterator=True, index_col=0, parse_dates=['start_date', 'end_date', 'forecast_time']):
        
    # append chunk to chunks list
    chunks.append(chunk)

    if chunk_counter == 1 or chunk_counter % math.ceil(num_chunks/10) == 0 or chunk_counter == num_chunks:
        print('\t\t[%s] finished chunk %s of %s' % (datetime.datetime.now().time(), chunk_counter, num_chunks))
    chunk_counter += 1

morning_commutes = pd.concat(chunks)
morning_commutes.reset_index(inplace=True, drop=True)
print('[%s] Complete!' % datetime.datetime.now().time())


print('[%s] Loading Evening Commute Trips Data...' % datetime.datetime.now().time())

evening_commutes = pd.DataFrame()
trip_data_file = '../../clean_data/bayareabikeshare/trip_data_evening_commutes.csv'

# Chunk Settings
chunks = []
chunk_counter = 1
chunksize = 10000
num_chunks = math.ceil(sum(1 for row in open(trip_data_file, 'r'))/chunksize)

# import file in chunks
for chunk in pd.read_csv(trip_data_file, chunksize=chunksize, iterator=True, index_col=0, parse_dates=['start_date', 'end_date', 'forecast_time']):
        
    # append chunk to chunks list
    chunks.append(chunk)

    if chunk_counter == 1 or chunk_counter % math.ceil(num_chunks/10) == 0 or chunk_counter == num_chunks:
        print('\t\t[%s] finished chunk %s of %s' % (datetime.datetime.now().time(), chunk_counter, num_chunks))
    chunk_counter += 1

evening_commutes = pd.concat(chunks)
evening_commutes.reset_index(inplace=True, drop=True)
print('[%s] Complete!' % datetime.datetime.now().time())

[16:14:21.841893] Loading Morning Commute Trips Data...
		[16:14:22.454640] finished chunk 1 of 26
		[16:14:22.876969] finished chunk 3 of 26
		[16:14:23.562311] finished chunk 6 of 26
		[16:14:24.267914] finished chunk 9 of 26
		[16:14:25.063869] finished chunk 12 of 26
		[16:14:25.934265] finished chunk 15 of 26
		[16:14:26.635970] finished chunk 18 of 26
		[16:14:28.378347] finished chunk 21 of 26
		[16:14:29.568379] finished chunk 24 of 26
		[16:14:30.176355] finished chunk 26 of 26
[16:14:30.593857] Complete!
[16:14:30.594075] Loading Evening Commute Trips Data...
		[16:14:31.315940] finished chunk 1 of 24
		[16:14:31.835464] finished chunk 3 of 24
		[16:14:32.459358] finished chunk 6 of 24
		[16:14:33.144471] finished chunk 9 of 24
		[16:14:33.750408] finished chunk 12 of 24
		[16:14:35.152462] finished chunk 15 of 24
		[16:14:36.849258] finished chunk 18 of 24
		[16:14:37.895029] finished chunk 21 of 24
		[16:14:38.595505] finished chunk 24 of 24
[16:14:38.941668] Complete!


In [5]:
def lowercase_summaries(df=None, columns=['daily_icon', 'daily_summary', 'hourly_icon', 'hourly_summary']):
    
    for col in columns:
        df[col].fillna('', inplace=True)
        df[col] = df[col].apply(lambda x: x.lower())

    return df
   
evening_commutes = lowercase_summaries(df=evening_commutes)
morning_commutes = lowercase_summaries(df=morning_commutes)


# Load DarkSky Data at Super Station

In [6]:
print('Started Loading Weather Data...')
darksky_data_file = '../../clean_data/darksky/sanfrancisco/station_69_darksky_cleaned.csv'
file_list = [darksky_data_file]

darksky = pd.DataFrame()

num_files = len(file_list)
chunks = []

for i, file in enumerate(file_list):

    chunk = pd.read_csv(file, index_col=0, parse_dates=['time_corrected'])
    
    
    chunks.append(chunk)
    
    if (i + 1) == 1 or (i + 1) % math.ceil(num_files/10) == 0 or (i + 1) == num_files:
        print('\t[%s] finished chunk %s of %s' % (datetime.datetime.now().time(), str(i+1).rjust(8), str(num_files).rjust(8)))

    
darksky = pd.concat(chunks)

darksky.drop_duplicates(inplace=True)

darksky.set_index('time_corrected', inplace=True)

darksky = lowercase_summaries(df=darksky)

print('Data Loaded Successfully!')

Started Loading Weather Data...
	[16:14:40.910011] finished chunk        1 of        1
Data Loaded Successfully!


# Plot Yearly Weather Trends

In [7]:
def plot_yearly_weather(data=darksky, category='precipIntensity', title='', label='', legend_loc=2, y_label='', show_legend=True):
    
    if label == '':
        label = category
    if title == '':
        title = label
        
    yearly_means = pd.DataFrame()
    plt.subplots(figsize=FIG_SIZE)
    for year in sorted(data.index.year.unique()):
        y = data[data.index.year == year].copy()

        days = y.groupby(y.index.week)[category].mean()

        ax = days.plot(linestyle='', marker='.', alpha=0.75, color='c', label=label.title())
   

        days = days.to_frame()
        days.columns = [str(year)]

        if yearly_means.shape[0] == 0:
            yearly_means = days
        else:
            yearly_means = yearly_means.merge(days, left_index=True, right_index=True, how='outer')

        
    yearly_means.index.rename('day', inplace=True)
    
    yearly_means['average'] = yearly_means.mean(axis=1, skipna=True)
    yearly_means['std'] = yearly_means.std(axis=1, skipna=True)
    
    yearly_means['upper_bound'] = yearly_means['average'] + yearly_means['std']
    yearly_means['lower_bound'] = yearly_means['average'] - yearly_means['std']
    
    if category == 'precipIntensity':
        yearly_means['lower_bound'] = yearly_means['lower_bound'].apply(lambda x: 0 if x < 0 else x)
    
    
    yearly_means['upper_bound'].plot(linestyle=':', marker='', alpha=0.5, color='b', ax=ax, label='Normal {} Range'.format(label.title()))
    yearly_means['average'].plot(linestyle='-', marker='', alpha=0.5, color='m', ax=ax, label='Mean {}'.format(label.title()))
    yearly_means['lower_bound'].plot(linestyle=':', marker='', alpha=0.5, color='b', ax=ax, label='')
    
    ax.set_title('Yearly {}'.format(title.title()), size=TITLE_FONT_SIZE, weight='bold')
    

    ax.set_ylabel(y_label, size=LABEL_FONT_SIZE, weight='bold')
    
    x_ticks = [x*(53/12)+(53/24) for x in range(0, len(month_labels))]
    x_markers = [x*(53/12) for x in range(0, len(month_labels)+1)]
    
    ax.set_xlabel('', size=TICK_FONT_SIZE)
    ax.set_xticks(x_ticks)
    ax.set_xticklabels(month_labels, size=TICK_FONT_SIZE)
    for x in x_markers:
        ax.axvline(x=x, linestyle=':', alpha=0.25, color='k')
    ax.grid(False)
    
    if show_legend == True:
        ax.legend(loc=legend_loc, frameon=True)
#     plt.show()
    plt.savefig('../../charts/darksky/yearly_trend_{}.png'.format(title.lower().replace(' ', '')))
    plt.close()
    
    results = {'title':title,
                'data':yearly_means}

    return results

yearly_temperature_stats = plot_yearly_weather(data=darksky, category='apparentTemperature', label='Temperature', y_label='Temperature (F)')
yearly_precipitation_stats = plot_yearly_weather(data=darksky, category='precipIntensity', label='Precipitation Intensity', y_label='Precipitation Intensity (in)', show_legend=False)

# Plot Daily Weather Trends

In [8]:
def plot_daily_weather(data=darksky, category='precipIntensity', title='', label='', legend_loc=2, y_label=''):
        
    if label == '':
        label = category
    if title == '':
        title = label
        
    daily_means = pd.DataFrame()
    plt.subplots(figsize=(15,8))
    for date in sorted(data.index.dayofyear.unique()):
        h = data[data.index.dayofyear == date].copy()                
        hours = h.groupby(h.index.hour)[category].mean()        
        ax = hours.plot(linestyle='', marker='.', alpha=0.75, color='c', label=label.title())

        hours = hours.to_frame()
        hours.columns = [str(date)]

        if daily_means.shape[0] == 0:
            daily_means = hours
        else:
            daily_means = daily_means.merge(hours, left_index=True, right_index=True, how='outer')

        
    daily_means.index.rename('hour', inplace=True)
    daily_means['average'] = daily_means.mean(axis=1, skipna=True)
    daily_means['std'] = daily_means.std(axis=1, skipna=True)
    
    daily_means['upper_bound'] = daily_means['average'] + 1*daily_means['std']
    daily_means['lower_bound'] = daily_means['average'] - 1*daily_means['std']
    
    if category == 'precipIntensity':
        daily_means['lower_bound'] = daily_means['lower_bound'].apply(lambda x: 0 if x < 0 else x)
    
    
    daily_means['upper_bound'].plot(linestyle=':', marker='', alpha=0.5, color='b', ax=ax, label='Normal {} Range'.format(label.title()))
    daily_means['average'].plot(linestyle='-', marker='', alpha=0.5, color='m', ax=ax, label='Mean {}'.format(label.title()))
    daily_means['lower_bound'].plot(linestyle=':', marker='', alpha=0.5, color='b', ax=ax, label='')
    
    ax.set_title('Hourly {}'.format(title.title()), size=TITLE_FONT_SIZE, weight='bold')
    
    ax.set_xlabel('Hour of Day', size=LABEL_FONT_SIZE, weight='bold')
    ax.set_ylabel(y_label, size=LABEL_FONT_SIZE, weight='bold')
    
    ax.set_xticks([x for x in sorted(data.index.hour.unique())])
    ax.set_xlabel('', size=TICK_FONT_SIZE)
    
    x_markers = [x+0.5 for x in range(-1, 24, 1)]
    for x in x_markers:
        ax.axvline(x=x, linestyle=':', alpha=0.25, color='k')
        
    ax.grid(False)
    
    ax.legend(loc=legend_loc, frameon=True)
#     plt.show()
    plt.savefig('../../charts/darksky/daily_trend_{}.png'.format(title.lower().replace(' ', '')))
    plt.close()
    
    results = {'title':title,
                'data':daily_means}

    return results

daily_temperature_stats = plot_daily_weather(data=darksky, category='apparentTemperature', label='Temperature', y_label='Temperature (F)')
daily_precipitation_stats = plot_daily_weather(data=darksky, category='precipIntensity', label='Precipitation Intensity', y_label='Precipitation Intensity (in)')
daily_humidity_stats = plot_daily_weather(data=darksky, category='humidity', legend_loc=3)

daily_precipprob_stats = plot_daily_weather(data=darksky, category='precipProbability', label='Precipitation Probability', y_label='Precipitation Probability')
daily_windspeed_stats = plot_daily_weather(data=darksky, category='windSpeed', label='Wind Speed', y_label='Wind Spped (mph)')

# Get list of Rainy Days

In [9]:
def tag_rainy_days(df=None):
    
    df = df.copy()
    
    try:
        df.reset_index(inplace=True, drop=False)
    except:
        df.reset_index(inplace=True, drop=True)

    df['is_rainy_day'] = False
    
    rain_index = df[(df.precipIntensity > 0.0) | 
                                      (df.precipType > 0) | 
                                      (df.precipProbability >= 0.49)].index
    
    df.iloc[rain_index, list(df.columns).index('is_rainy_day')] = 1
    
    # tag days based on summary
    summary_columns   = ['daily_icon', 'daily_summary', 'hourly_icon', 'hourly_summary']

    for col in summary_columns:
        df['is_rainy_day'] = np.where(df[col].str.contains("rain", case=False, na=False), True, df['is_rainy_day'])
    
    df.set_index('time_corrected', inplace=True, drop=True)
    return df

In [10]:
darksky = tag_rainy_days(darksky)

In [11]:
print('{} DarkSky Hours'.format(str(darksky.shape[0])).rjust(6))
print('{} DarkSky Rainy Hours'.format(str(darksky[darksky.is_rainy_day == True].shape[0])).rjust(6))
print('{} DarkSky Not Rainy Hours'.format(str(darksky[darksky.is_rainy_day == False].shape[0])).rjust(6))

26256 DarkSky Hours
2156 DarkSky Rainy Hours
24100 DarkSky Not Rainy Hours


# Identify and Tag `Rainy` and `Dry` Commute Days
<ul> <b>Rainy Conditions</b> are if any of the following are met at trip start date and time
    <li>Precipitation Intensity is Greater than 0.0</li>
    <li>Precipitation Type is Rain</li>
    <li>Precipitation Probability is Greater than 0.59</li>
</ul>

In [12]:
def tag_rainy_trips(trip_df=None):
    df = trip_df.copy()    
    df['is_rainy_day'] = False
    
    rain_index = df[(df.precipIntensity > 0.0) | 
                                      (df.precipType > 0) | 
                                      (df.precipProbability >= 0.49)].index
    
    df.iloc[rain_index, list(df.columns).index('is_rainy_day')] = 1
    
    # tag days based on summary
    summary_columns   = ['daily_icon', 'daily_summary', 'hourly_icon', 'hourly_summary']

    for col in summary_columns:
        df['is_rainy_day'] = np.where(df[col].str.contains("rain", case=False, na=False), True, df['is_rainy_day'])
    
    return df

In [13]:
morning_commutes = tag_rainy_trips(trip_df=morning_commutes)
evening_commutes = tag_rainy_trips(trip_df=evening_commutes)

In [14]:
print('{} Morning Commutes'.format(str(morning_commutes.shape[0])).rjust(6))
print('{} Morning Commutes Rainy'.format(str(morning_commutes[morning_commutes.is_rainy_day == True].shape[0])).rjust(6))
print('{} Morning Commutes Not Rainy'.format(str(morning_commutes[morning_commutes.is_rainy_day == False].shape[0])).rjust(6))
print()
print('{} Evening Commutes'.format(str(evening_commutes.shape[0])).rjust(6))
print('{} Evening Commutes Rainy'.format(str(evening_commutes[evening_commutes.is_rainy_day == True].shape[0])).rjust(6))
print('{} Evening Commutes Not Rainy'.format(str(evening_commutes[evening_commutes.is_rainy_day == False].shape[0])).rjust(6))

255117 Morning Commutes
13559 Morning Commutes Rainy
241558 Morning Commutes Not Rainy

237827 Evening Commutes
12947 Evening Commutes Rainy
224880 Evening Commutes Not Rainy


# Identify and Tag `Cold` and `Hot` Commutes
<ul>
    <li><b>Cold Conditions</b> are trips starting when the temperature more than two standard deviations below the mean for that day of the year</li>
    <li><b>Hot Conditions</b> are trips starting when the temperature more than two standard deviations above the mean for that day of the year</li>
    <li><b>Normal Conditions</b> are trips starting when the temperature neither `Cold` nor `Hot`</li>
</ul>

In [15]:
dtemps = daily_temperature_stats['data'].iloc[:,:-4].copy()
dtemps.index.rename('dayofyear', inplace=True)
dtemps = dtemps.transpose()

dtemps['mean'] = dtemps.mean(axis=1)
dtemps['std'] = dtemps.std(axis=1)

dtemps['lower_bound'] = dtemps['mean'] - 1*dtemps['std']
dtemps['upper_bound'] = dtemps['mean'] + 1*dtemps['std']

dtemps.head(30)

dayofyear,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,mean,std,lower_bound,upper_bound
1,45.863333,44.643333,44.026667,44.130000,42.930000,41.800000,41.773333,42.250000,41.750000,43.280000,...,48.890000,47.143333,46.433333,46.596667,45.966667,45.083333,46.089861,2.978125,43.111736,49.067986
2,44.526667,43.296667,43.390000,44.010000,42.470000,42.306667,42.413333,44.010000,42.636667,44.853333,...,50.990000,50.440000,49.893333,49.033333,47.663333,47.650000,47.409583,3.768188,43.641396,51.177771
3,47.200000,46.960000,46.710000,45.780000,45.356667,45.043333,43.863333,44.180000,44.640000,45.590000,...,52.973333,51.780000,51.026667,50.210000,49.366667,48.450000,48.909167,3.532637,45.376529,52.441804
4,48.026667,47.143333,46.536667,46.016667,45.776667,45.246667,45.233333,44.126667,44.476667,46.580000,...,54.766667,53.440000,52.753333,52.223333,50.796667,50.850000,50.134861,4.079472,46.055389,54.214333
5,50.123333,50.203333,49.650000,49.730000,49.510000,50.076667,49.916667,49.406667,50.400000,53.100000,...,56.330000,55.086667,53.886667,53.006667,52.580000,51.416667,53.026667,2.830409,50.196257,55.857076
6,50.666667,50.306667,49.400000,47.790000,47.640000,45.100000,45.636667,46.653333,46.380000,50.463333,...,55.230000,54.480000,53.136667,51.783333,51.556667,51.070000,51.423472,3.441039,47.982433,54.864512
7,50.186667,49.296667,49.163333,48.920000,48.790000,48.930000,48.553333,48.800000,49.223333,50.083333,...,54.530000,54.500000,53.340000,52.923333,52.400000,52.036667,52.065833,2.710634,49.355199,54.776467
8,51.243333,50.623333,50.143333,50.046667,50.086667,49.346667,49.433333,49.543333,50.080000,50.963333,...,53.240000,53.436667,52.906667,52.656667,52.140000,51.666667,51.882222,1.679531,50.202691,53.561753
9,51.420000,51.480000,51.133333,50.093333,49.916667,49.776667,49.886667,49.970000,51.213333,51.943333,...,54.653333,53.836667,53.286667,52.956667,52.526667,52.186667,52.695972,2.067962,50.628011,54.763934
10,51.800000,50.653333,50.703333,50.260000,50.223333,49.756667,48.490000,48.520000,48.300000,51.143333,...,53.726667,53.196667,52.550000,52.303333,51.386667,51.230000,51.893194,2.084756,49.808439,53.977950


In [16]:
def temperature_classifier(row, temperature_data, hot_cold='hot'):
    temp_at_start = row.apparentTemperature    
    
    if hot_cold == 'hot':
        if temp_at_start > temperature_data.iloc[row.start_date.dayofyear-1,list(temperature_data.columns).index('upper_bound')] or temp_at_start >= 75:
            return True
        else:
            return False
    
    if hot_cold == 'cold':
        if temp_at_start < temperature_data.iloc[row.start_date.dayofyear-1,list(temperature_data.columns).index('lower_bound')] or temp_at_start <= 40:
            return True
        else:
            return False

In [17]:
def tag_temperature_category(trip_df=None):
    df = trip_df.copy()

    # compare trip apparentTemperature to mean for that day of the year
    df['is_hot']  = df.apply(lambda row: temperature_classifier(row, dtemps, hot_cold='hot'), axis=1)
    df['is_cold'] = df.apply(lambda row: temperature_classifier(row, dtemps, hot_cold='cold'), axis=1)
    
    return df
    

## Tag Trips `is_hot` and `is_cold`

In [18]:
morning_commutes = tag_temperature_category(trip_df=morning_commutes)
evening_commutes = tag_temperature_category(trip_df=evening_commutes)

In [19]:
ss_morning_commutes = morning_commutes[(morning_commutes.start_station_id.isin(super_stations)) | 
                                       (morning_commutes.end_station_id.isin(super_stations))].copy()
ss_evening_commutes = evening_commutes[(evening_commutes.start_station_id.isin(super_stations)) | 
                                       (evening_commutes.end_station_id.isin(super_stations))].copy()

In [20]:
def plot_hot_cold_histograms(morning_commutes=morning_commutes, 
                                evening_commutes=evening_commutes,
                                bins=100, 
                                y_ticks = [y for y in range(0, 12500, 2500)], 
                                x_ticks = [x for x in range(35, 95, 5)], 
                                has_grid = False, 
                                legend_loc = 1, 
                                frameon=True,
                                file_prefix='all_stations_',
                                write=False):

    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(GRID_DIMS, GRID_DIMS*.5))

    morning_commutes[morning_commutes.is_cold == True].apparentTemperature.hist(ax=axes[0,0], bins=bins, color='b', alpha=0.85)
    morning_commutes[morning_commutes.is_cold == False].apparentTemperature.hist(ax=axes[0,0], bins=bins, color='k', alpha=0.35)
    axes[0,0].set_yticks(y_ticks)
    axes[0,0].set_xticks(x_ticks)
    axes[0,0].set_title('Cold Morning Commutes')
    axes[0,0].legend(['{:2.2f}% Cold'.format(morning_commutes[morning_commutes.is_cold == True].shape[0]/morning_commutes.shape[0]*100), 
                      '{:2.2f}% Not Cold'.format(morning_commutes[morning_commutes.is_cold == False].shape[0]/morning_commutes.shape[0]*100)], loc=legend_loc, frameon=frameon)
    axes[0,0].grid(has_grid)


    morning_commutes[morning_commutes.is_hot == True].apparentTemperature.hist(ax=axes[0,1], bins=bins, color='r', alpha=0.85)
    morning_commutes[morning_commutes.is_hot == False].apparentTemperature.hist(ax=axes[0,1], bins=bins, color='k', alpha=0.35)
    axes[0,1].set_yticks(y_ticks)
    axes[0,1].set_xticks(x_ticks)
    axes[0,1].set_title('Hot Morning Commutes')
    axes[0,1].legend(['{:2.2f}% Hot'.format(morning_commutes[morning_commutes.is_hot == True].shape[0]/morning_commutes.shape[0]*100), 
                      '{:2.2f}% Not Hot'.format(morning_commutes[morning_commutes.is_hot == False].shape[0]/morning_commutes.shape[0]*100)], loc=legend_loc, frameon=frameon)
    axes[0,1].grid(has_grid)


    evening_commutes[evening_commutes.is_cold == True].apparentTemperature.hist(ax=axes[1,0], bins=bins, color='g', alpha=0.85)
    evening_commutes[evening_commutes.is_cold == False].apparentTemperature.hist(ax=axes[1,0], bins=bins, color='k', alpha=0.35)
    axes[1,0].set_yticks(y_ticks)
    axes[1,0].set_xticks(x_ticks)
    axes[1,0].set_title('Cold Evening Commutes')
    axes[1,0].legend(['{:2.2f}% Cold'.format(evening_commutes[evening_commutes.is_cold == True].shape[0]/evening_commutes.shape[0]*100), 
                      '{:2.2f}% Not Cold'.format(evening_commutes[evening_commutes.is_cold == False].shape[0]/evening_commutes.shape[0]*100)], loc=legend_loc, frameon=frameon)
    axes[1,0].grid(has_grid)


    evening_commutes[evening_commutes.is_hot == True].apparentTemperature.hist(ax=axes[1,1], bins=bins, color='m', alpha=0.85)
    evening_commutes[evening_commutes.is_hot == False].apparentTemperature.hist(ax=axes[1,1], bins=bins, color='k', alpha=0.35)
    axes[1,1].set_yticks(y_ticks)
    axes[1,1].set_xticks(x_ticks)
    axes[1,1].set_title('Hot Evening Commutes')
    axes[1,1].legend(['{:2.2f}% Hot'.format(evening_commutes[evening_commutes.is_hot == True].shape[0]/evening_commutes.shape[0]*100), 
                      '{:2.2f}% Not Hot'.format(evening_commutes[evening_commutes.is_hot == False].shape[0]/evening_commutes.shape[0]*100)], loc=legend_loc, frameon=frameon)
    axes[1,1].grid(has_grid)

    plt.tight_layout()
    file_name = '../../charts/statistical_analysis/{}_hot_and_cold_commutes.png'.format(file_prefix)
    
    if write:
        plt.savefig(file_name)
    else:
        plt.show()
    print(file_name)
    plt.close()

In [21]:
plot_hot_cold_histograms(morning_commutes=morning_commutes, 
                                evening_commutes=evening_commutes,
                                bins=100, 
                                y_ticks = [y for y in range(0, 12500, 2500)], 
                                x_ticks = [x for x in range(35, 95, 5)], 
                                has_grid = False, 
                                legend_loc = 1, 
                                frameon=True,
                                file_prefix='1dev_all_stations',
                                write=DO_WRITE_CHARTS)

../../charts/statistical_analysis/1dev_all_stations_hot_and_cold_commutes.png


In [22]:
plot_hot_cold_histograms(morning_commutes=ss_morning_commutes, 
                                evening_commutes=ss_evening_commutes,
                                bins=100, 
                                y_ticks = [y for y in range(0, 4000, 1000)], 
                                x_ticks = [x for x in range(35, 95, 5)], 
                                has_grid = False, 
                                legend_loc = 1, 
                                frameon=True,
                                file_prefix='1dev_superstation',
                                write=DO_WRITE_CHARTS)

../../charts/statistical_analysis/1dev_superstation_hot_and_cold_commutes.png


# RankSum and Two Sample T Tests

In [23]:
from scipy.stats import ranksums
from scipy.stats import ttest_ind

In [24]:
def ranksums_test(dfA, dfB, a_label='', b_label='', interval_label='dates'):
    
    if dfA.shape[0] > 0:
        A_mean = dfA.mean()
    else:
        A_mean = 0

    if dfB.shape[0] > 0:
        B_mean = dfB.mean()
    else:
        B_mean = 0

    diff_of_means = A_mean - B_mean
    
    if A_mean > 0 or B_mean > 0:
        # perform ranksums test
        z, z_p = ranksums(dfA, dfB)
        t, t_p = ttest_ind(dfA, dfB)
        drop_share = (A_mean - B_mean) / B_mean * 100.
        
    else:
        z, z_p = [0, 0]
        t, t_p = [0, 0]
        drop_share = 0
    
    # cleanup drop share column
    if drop_share == np.inf or drop_share == -np.inf:
        drop_share = 0
    
    if dfA.shape[0] <= 1 or dfB.shape[0] <= 1:
        drop_share = 0
    
    a_to_b_ratio = dfA.shape[0] / dfB.shape[0]
    
    if drop_share < -400:
        sig_drop_share = 100
    if drop_share > 400:
        sig_drop_share = 100

    sig_diff_of_means = diff_of_means
    sig_drop_share = drop_share
    # if there are not enough cat A days, the drop share is not very meaningful
    if a_to_b_ratio <= 0.031:
        drop_share = 0
        sig_diff_of_means = 0
        sig_drop_share = 0
    

    z_can_reject = False
    t_can_reject = False

    if z_p <= 0.05:
        z_can_reject = True
    if t_p <= 0.05:
        t_can_reject = True
        
    if z_can_reject != True and t_can_reject != True:
       sig_diff_of_means = 0 
       sig_drop_share = 0
        
    results = {'z_score':z, 
                'z_p_value':z_p,
                't_score':t, 
                't_p_value':t_p,
                '{}_{}'.format(a_label.lower(), interval_label):dfA.shape[0], 
                '{}_{}'.format(b_label.lower(), interval_label):dfB.shape[0], 
                '{}_mean'.format(a_label.lower()):A_mean, 
                '{}_mean'.format(b_label.lower()):B_mean, 
                'diff_of_means':diff_of_means,
                'sig_diff_of_means':sig_diff_of_means,
                'drop_share':drop_share,
               'sig_drop_share':sig_drop_share,
                'z_can_reject':z_can_reject,
                't_can_reject':t_can_reject,
                'day_ratio':a_to_b_ratio}
    
    return results
    

In [25]:
def monthly_tests(df=ss_morning_commutes, test_cat='Cold'):
    chunks = []

    for month in sorted(df.start_date.dt.month.unique()):
        month_data = df[df.start_date.dt.month == month].copy()

        if test_cat.lower() == 'cold':
            # Morning Commutes
            COLD = month_data[month_data.is_cold == True].copy()
            COLD = COLD.groupby(COLD.start_date.dt.date)['trip_id'].count()

            NOTCOLD = month_data[month_data.is_cold == False].copy()
            NOTCOLD = NOTCOLD.groupby(NOTCOLD.start_date.dt.date)['trip_id'].count()

            R = ranksums_test(COLD, NOTCOLD, a_label='Cold', b_label='Normal', interval_label='dates')

        if test_cat.lower() == 'hot':
            # Morning Commutes
            HOT = month_data[month_data.is_hot == True].copy()
            HOT = HOT.groupby(HOT.start_date.dt.date)['trip_id'].count()

            NOTHOT = month_data[month_data.is_hot == False].copy()
            NOTHOT = NOTHOT.groupby(NOTHOT.start_date.dt.date)['trip_id'].count()

            R = ranksums_test(HOT, NOTHOT, a_label='Hot', b_label='Normal', interval_label='dates')

        if test_cat.lower() == 'rain':
            # Morning Commutes
            RAINY = month_data[month_data.is_rainy_day == True].copy()
            RAINY = RAINY.groupby(RAINY.start_date.dt.date)['trip_id'].count()

            DRY = month_data[month_data.is_rainy_day == False].copy()
            DRY = DRY.groupby(DRY.start_date.dt.date)['trip_id'].count()

            R = ranksums_test(RAINY, DRY, a_label='Rainy', b_label='Dry', interval_label='dates')
            
        chunks.append(R)

    monthly_test = pd.DataFrame(chunks)
    monthly_test.fillna(0, inplace=True)

    return monthly_test

# Plot P Values of Z and T Tests

In [26]:
def plot_pvalue_test(test_results=None, a='Cold', b='Normal', title='morning', write=False, upper_legend_loc=3, lower_legend_loc=3):

    ax = test_results[['z_p_value']].plot(kind='bar', figsize=FIG_SIZE_SHORT, color=sub_color, alpha=0.35)
    test_results[['t_p_value']].plot(kind='bar', ax=ax, color=cust_color, alpha=0.35)

    ax.legend(['RankSum', 'Two Sample T'], loc=upper_legend_loc, frameon=True)
    
    plt.axhline(y=0.05, xmin=-10, xmax=12, hold=None)
    ax.set_ylim([0, 0.1])
    ax.set_xticks(test_results.index)
    ax.set_xticklabels([month_labels[x] for x in test_results.index], rotation=0)
    title1 = 'P Value {} vs {} Mean Daily {} Commuter Trips'.format(a.title(), b.title(), title.title())
    ax.set_title(title1, size=TITLE_FONT_SIZE)

    save_path = '../../charts/statistical_analysis/{}_{}_commute_pvalue.png'.format(title.lower(), a.lower())
    print(save_path)
    
    if write:
        plt.savefig(save_path)
    else:
        plt.show()
    plt.close()

    # DIFF OF MEANS
    ax = test_results[['sig_diff_of_means']].plot(kind='bar', figsize=FIG_SIZE_SHORT, color=cust_color, alpha=0.35)
    ax.set_xticks(range(0, len(test_results.index)))
    ax.set_xticklabels([month_labels[x] for x in test_results.index], rotation=0)

    ax.set_title('{} Statistically Significant Difference of Means'.format(title.title()), size=TITLE_FONT_SIZE)
    
    plt.legend(['Difference in Daily Mean Trips'], loc=lower_legend_loc, frameon=True)

    save_path = '../../charts/statistical_analysis/{}_{}_commute_diff.png'.format(title.lower(), a.lower())
    print(save_path)
    
    if write:
        plt.savefig(save_path)
    else:
        plt.show()
    plt.close()
    
    # Drop Share
    ax = test_results[['sig_drop_share']].plot(kind='bar', figsize=FIG_SIZE_SHORT, color=cust_color, alpha=0.35)
    ax.set_xticks(range(0, len(test_results.index)))
    ax.set_xticklabels([month_labels[x] for x in test_results.index], rotation=0)

    ax.set_title('{} Statistically Significant Percentage Change'.format(title.title()), size=TITLE_FONT_SIZE)
    
    plt.legend(['Drop Percentage'], loc=lower_legend_loc, frameon=True)

    save_path = '../../charts/statistical_analysis/{}_{}_commute_drop.png'.format(title.lower(), a.lower())
    print(save_path)
    
    if write:
        plt.savefig(save_path)
    else:
        plt.show()
    plt.close()



# Impact of Cold on Morning Commuter Traffic


<div class="alert alert-info">

<p><b>1a. Morning Cold Commutes</b></p>

<p>A <b>Wilcoxon Rank-Sum Statistic Test</b> is appropriate for this problem as we are trying to see a difference between two sample means from data sets with very different values.</p>

<ul>
    <li>$H$o : Morning Commuter Mean Number of trips on Cold Days = Morning Commuter Mean Number of trips on Normal Days</li>
    <li>$H$a : Morning Commuter Mean Number of trips on Cold Days ≠ Morning Commuter Mean Number of trips on Normal Days</li>
</ul>
</div>

In [27]:
morning_cold_commute_t_test =  monthly_tests(df=ss_morning_commutes, test_cat='cold')
morning_cold_commute_t_test

,cold_dates,cold_mean,day_ratio,diff_of_means,drop_share,normal_dates,normal_mean,sig_diff_of_means,sig_drop_share,t_can_reject,t_p_value,t_score,z_can_reject,z_p_value,z_score
0,38,97.000000,0.775510,20.632653,27.017638,49,76.367347,0.000000,0.000000,False,0.059965,1.906498,False,0.071639,1.801406
1,43,81.604651,0.977273,-9.258985,-10.189979,44,90.863636,0.000000,0.000000,False,0.414928,-0.819262,False,0.280952,-1.078183
2,47,97.936170,1.044444,9.180615,10.343707,45,88.755556,0.000000,0.000000,False,0.405761,0.835300,False,0.486956,0.695159
3,41,97.780488,0.891304,-10.545599,-9.735050,46,108.326087,0.000000,0.000000,False,0.356950,-0.926227,False,0.309571,-1.016123
4,45,89.755556,0.882353,-8.264052,-8.431020,51,98.019608,0.000000,0.000000,False,0.510839,-0.660046,False,0.702622,-0.381783
5,44,86.136364,0.800000,-19.900000,-18.767147,55,106.036364,0.000000,0.000000,False,0.073030,-1.812326,False,0.091698,-1.686508
6,35,98.400000,0.686275,-15.031373,-13.251513,51,113.431373,0.000000,0.000000,False,0.205661,-1.275465,False,0.237080,-1.182318
7,39,110.820513,0.866667,-11.868376,-9.673554,45,122.688889,0.000000,0.000000,False,0.344412,-0.950975,False,0.907180,-0.116597
8,40,90.250000,0.816327,11.209184,14.181513,49,79.040816,0.000000,0.000000,False,0.379683,0.882974,False,0.294879,1.047478
9,33,91.848485,0.702128,-31.513217,-25.545381,47,123.361702,-31.513217,-25.545381,True,0.004289,-2.942252,True,0.001595,-3.156783


In [28]:
plot_pvalue_test(test_results=morning_cold_commute_t_test, a='Cold', b='Normal',
                 title='morning', write=DO_WRITE_CHARTS)

../../charts/statistical_analysis/morning_cold_commute_pvalue.png
../../charts/statistical_analysis/morning_cold_commute_diff.png
../../charts/statistical_analysis/morning_cold_commute_drop.png


# Impact of Heat on Morning Commuter Traffic

<div class="alert alert-info">

<p><b>1a. Morning Hot Commutes</b></p>

<p>A <b>Wilcoxon Rank-Sum Statistic Test</b> is appropriate for this problem as we are trying to see a difference between two sample means from data sets with very different values.</p>

<ul>
    <li>$H$o : Morning Commuter Mean Number of trips on Hot Days = Morning Commuter Mean Number of trips on Normal Days</li>
    <li>$H$a : Morning Commuter Mean Number of trips on Hot Days ≠ Morning Commuter Mean Number of trips on Normal Days</li>
</ul>
</div>

In [29]:
morning_hot_commute_t_test =  monthly_tests(df=ss_morning_commutes, test_cat='hot')
morning_hot_commute_t_test

,day_ratio,diff_of_means,drop_share,hot_dates,hot_mean,normal_dates,normal_mean,sig_diff_of_means,sig_drop_share,t_can_reject,t_p_value,t_score,z_can_reject,z_p_value,z_score
0,0.078125,-69.271875,-61.481071,5,43.400000,64,112.671875,-69.271875,-61.481071,True,1.312487e-03,-3.354401,True,0.014150,-2.453423
1,0.118644,-19.527845,-16.861450,7,96.285714,59,115.813559,0.000000,0.000000,False,2.967475e-01,-1.052017,False,0.532142,-0.624740
2,0.045455,-102.378788,-79.335447,3,26.666667,66,129.045455,-102.378788,-79.335447,True,1.176936e-06,-5.343758,True,0.003579,-2.913025
3,0.093750,-67.218750,-50.081490,6,67.000000,64,134.218750,-67.218750,-50.081490,True,3.093805e-05,-4.465225,True,0.012540,-2.496565
4,0.061538,-72.700000,-53.772189,4,62.500000,65,135.200000,-72.700000,-53.772189,True,1.787283e-03,-3.253570,True,0.008174,-2.644796
5,0.060606,-128.022727,-88.430141,4,16.750000,66,144.772727,-128.022727,-88.430141,True,6.247501e-13,-8.851984,True,0.001099,-3.263993
6,0.078125,-69.381250,-49.993245,5,69.400000,64,138.781250,-69.381250,-49.993245,True,5.426574e-05,-4.312469,False,0.091100,-1.689622
7,0.102941,-78.258403,-56.486269,7,60.285714,68,138.544118,-78.258403,-56.486269,True,3.775352e-05,-4.388978,True,0.001482,-3.178160
8,0.093750,-81.921875,-71.910575,6,32.000000,64,113.921875,-81.921875,-71.910575,True,8.050773e-04,-3.508043,True,0.001234,-3.230848
9,0.134328,-83.756219,-66.425979,9,42.333333,67,126.089552,-83.756219,-66.425979,True,4.237518e-08,-6.111077,True,0.000036,-4.131614


In [30]:
plot_pvalue_test(test_results=morning_hot_commute_t_test, a='Hot', b='Normal',
                 title='morning', write=DO_WRITE_CHARTS, upper_legend_loc=2, lower_legend_loc=4)

../../charts/statistical_analysis/morning_hot_commute_pvalue.png
../../charts/statistical_analysis/morning_hot_commute_diff.png
../../charts/statistical_analysis/morning_hot_commute_drop.png


# Impact of Cold on Evening Commuter Traffic


<div class="alert alert-info">

<p><b>2a. Evening Cold Commutes</b></p>

<p>A <b>Wilcoxon Rank-Sum Statistic Test</b> is appropriate for this problem as we are trying to see a difference between two sample means from data sets with very different values.</p>

<ul>
    <li>$H$o : Evening Commuter Mean Number of trips on Cold Days = Evening Commuter Mean Number of trips on Normal Days</li>
    <li>$H$a : Evening Commuter Mean Number of trips on Cold Days ≠ Evening Commuter Mean Number of trips on Normal Days</li>
</ul>
</div>

In [31]:
evening_cold_commute_t_test =  monthly_tests(df=ss_evening_commutes, test_cat='cold')
evening_cold_commute_t_test

/Users/samgutentag/anaconda/envs/bikeshare/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3146: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/Users/samgutentag/anaconda/envs/bikeshare/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/samgutentag/anaconda/envs/bikeshare/lib/python3.6/site-packages/scipy/stats/stats.py:4956: RuntimeWarning: invalid value encountered in double_scalars
  z = (s - expected) / np.sqrt(n1*n2*(n1+n2+1)/12.0)


,cold_dates,cold_mean,day_ratio,diff_of_means,drop_share,normal_dates,normal_mean,sig_diff_of_means,sig_drop_share,t_can_reject,t_p_value,t_score,z_can_reject,z_p_value,z_score
0,3,1.000000,0.046875,-117.203125,-99.153999,64,118.203125,-117.203125,-99.153999,True,2.125613e-06,-5.205059,True,0.003609,-2.910428
1,10,36.300000,0.172414,-83.217241,-69.627813,58,119.517241,-83.217241,-69.627813,True,1.156868e-07,-5.944140,True,0.000020,-4.268438
2,3,87.666667,0.046875,-45.145833,-33.992157,64,132.812500,-45.145833,-33.992157,True,2.072506e-02,-2.370789,True,0.031358,-2.152504
3,6,37.166667,0.092308,-95.587179,-72.003322,65,132.753846,-95.587179,-72.003322,True,1.294586e-07,-5.885538,True,0.001013,-3.286920
4,5,37.400000,0.076923,-96.800000,-72.131148,65,134.200000,-96.800000,-72.131148,True,4.610630e-06,-4.979906,True,0.000906,-3.318049
5,1,44.000000,0.015152,-103.606061,0.000000,66,147.606061,0.000000,0.000000,False,0.000000e+00,0.000000,False,0.097989,-1.654681
6,0,0.000000,0.000000,-138.552239,0.000000,67,138.552239,0.000000,0.000000,False,0.000000e+00,0.000000,False,0.000000,0.000000
7,5,7.200000,0.074627,-142.352239,-95.185629,67,149.552239,-142.352239,-95.185629,True,3.644285e-15,-10.022186,True,0.000257,-3.655028
8,2,55.000000,0.030769,-61.723077,0.000000,65,116.723077,0.000000,0.000000,False,1.518952e-01,-1.449898,False,0.161494,-1.400065
9,8,54.000000,0.117647,-70.147059,-56.503198,68,124.147059,-70.147059,-56.503198,True,3.423231e-04,-3.755447,True,0.000855,-3.334357


In [32]:
plot_pvalue_test(test_results=evening_cold_commute_t_test, a='Cold', b='Normal',
                 title='evening', write=DO_WRITE_CHARTS, lower_legend_loc=1, upper_legend_loc=4)

../../charts/statistical_analysis/evening_cold_commute_pvalue.png
../../charts/statistical_analysis/evening_cold_commute_diff.png
../../charts/statistical_analysis/evening_cold_commute_drop.png


# Impact of Heat on Evening Commuter Traffic


<div class="alert alert-info">

<p><b>2b. Evening Hot Commutes</b></p>

<p>A <b>Wilcoxon Rank-Sum Statistic Test</b> is appropriate for this problem as we are trying to see a difference between two sample means from data sets with very different values.</p>

<ul>
    <li>$H$o : Evening Commuter Mean Number of trips on Cold Days = Evening Commuter Mean Number of trips on Normal Days</li>
    <li>$H$a : Evening Commuter Mean Number of trips on Cold Days ≠ Evening Commuter Mean Number of trips on Normal Days</li>
</ul>
</div>

In [33]:
evening_hot_commute_t_test =  monthly_tests(df=ss_evening_commutes, test_cat='hot')
evening_hot_commute_t_test

,day_ratio,diff_of_means,drop_share,hot_dates,hot_mean,normal_dates,normal_mean,sig_diff_of_means,sig_drop_share,t_can_reject,t_p_value,t_score,z_can_reject,z_p_value,z_score
0,1.146341,21.372081,28.654523,47,95.957447,41,74.585366,21.372081,28.654523,True,0.044735,2.036906,False,0.061549,1.869529
1,1.250000,42.511111,63.979933,45,108.955556,36,66.444444,42.511111,63.979933,True,0.000388,3.707031,True,0.000453,3.507136
2,1.333333,34.576923,45.175879,52,111.115385,39,76.538462,34.576923,45.175879,True,0.002130,3.163989,True,0.002851,2.983360
3,1.069767,18.574317,20.670177,46,108.434783,43,89.860465,0.000000,0.000000,False,0.105936,1.633700,False,0.115890,1.572262
4,1.102564,3.407275,3.188189,43,110.279070,39,106.871795,0.000000,0.000000,False,0.791843,0.264802,False,0.933402,-0.083566
5,1.184211,15.742690,14.394182,45,125.111111,38,109.368421,0.000000,0.000000,False,0.210084,1.263360,False,0.163356,1.393872
6,1.260870,36.054723,52.138235,58,105.206897,46,69.152174,36.054723,52.138235,True,0.001351,3.295677,True,0.002339,3.043379
7,1.410256,12.146387,12.161979,55,112.018182,39,99.871795,0.000000,0.000000,False,0.292783,1.058094,False,0.235774,1.185615
8,1.282051,14.095897,17.941906,50,92.660000,39,78.564103,0.000000,0.000000,False,0.283480,1.079202,False,0.360863,0.913723
9,1.361111,22.232993,24.276244,49,113.816327,36,91.583333,0.000000,0.000000,False,0.075669,1.798904,False,0.067595,1.827696


In [34]:
plot_pvalue_test(test_results=evening_hot_commute_t_test, a='Hot', b='Normal',
                 title='evening', write=DO_WRITE_CHARTS, lower_legend_loc=4, upper_legend_loc=4)

../../charts/statistical_analysis/evening_hot_commute_pvalue.png
../../charts/statistical_analysis/evening_hot_commute_diff.png
../../charts/statistical_analysis/evening_hot_commute_drop.png


# RAIN

In [35]:
def plot_rain_histograms(morning_commutes=morning_commutes, 
                                evening_commutes=evening_commutes,
                                bins=100, 
                                y_ticks = [y for y in range(0, 15*61)], 
                                x_ticks = [x for x in range(0, 15*61)], 
                                has_grid = False, 
                                legend_loc = 1, 
                                frameon=True,
                                title_prefix='',
                                file_prefix='all_stations_',
                                write=False):

    fig, axes = plt.subplots(nrows=12, ncols=2, figsize=(GRID_DIMS, GRID_DIMS*3))

    # 15 minute ride in seconds
    duration_cutoff = 15*60
    
    for i, month in enumerate(month_labels):
        
        MORNING = morning_commutes[morning_commutes.start_date.dt.month == i+1].copy()
        MORNING[(MORNING.is_rainy_day == True) & (MORNING.duration <= duration_cutoff)].start_date.dt.day.hist(ax=axes[i, 0], bins=bins, color='b', alpha=0.85)
        MORNING[(MORNING.is_rainy_day == False) & (MORNING.duration <= duration_cutoff)].start_date.dt.day.hist(ax=axes[i, 0], bins=bins, color='k', alpha=0.35)
        axes[i, 0].set_xticks([x for x in range(0, MORNING.start_date.dt.day.max(), 5)])
        axes[i, 0].set_title('{} {} Rainy Morning Commutes'.format(month_labels_full[i].title(), title_prefix.title()).replace('  ', ' '))
        try:
            rain_share = MORNING[MORNING.is_rainy_day == True].shape[0]/MORNING.shape[0]*100
        except:
            rain_share = 0
        try:
            dry_share = MORNING[MORNING.is_rainy_day == False].shape[0]/MORNING.shape[0]*100
        except:
            dry_share = 0
        axes[i, 0].legend(['{:2.2f}%'.format(rain_share), 
                          '{:2.2f}%'.format(dry_share)], loc=legend_loc, frameon=frameon)
        axes[i, 0].grid(has_grid)

        EVENING = evening_commutes[evening_commutes.start_date.dt.month == i+1].copy()
        EVENING[(EVENING.is_rainy_day == True) & (EVENING.duration <= duration_cutoff)].start_date.dt.day.hist(ax=axes[i, 1], bins=bins, color='g', alpha=0.85)
        EVENING[(EVENING.is_rainy_day == False) & (EVENING.duration <= duration_cutoff)].start_date.dt.day.hist(ax=axes[i, 1], bins=bins, color='k', alpha=0.35)
        axes[i, 1].set_xticks([x for x in range(0, EVENING.start_date.dt.day.max(), 5)])
        axes[i, 1].set_title('{} {} Rainy Evening Commutes'.format(month_labels_full[i].title(), title_prefix.title()).replace('  ', ' '))
        try:
            rain_share = EVENING[EVENING.is_rainy_day == True].shape[0]/EVENING.shape[0]*100
        except:
            rain_share = 0
        try:
            dry_share = EVENING[EVENING.is_rainy_day == False].shape[0]/EVENING.shape[0]*100
        except:
            dry_share = 0
        axes[i, 1].legend(['{:2.2f}%'.format(rain_share), 
                          '{:2.2f}%'.format(dry_share)], loc=legend_loc, frameon=frameon)
        axes[i, 1].grid(has_grid)


    plt.tight_layout()
    file_name = '../../charts/statistical_analysis/rain_{} {}_commutes.png'.format(file_prefix, title_prefix.lower()).replace(' ', '_').replace('__', '_')
    
    if write:
        plt.savefig(file_name)
    else:
        plt.show()
    print(file_name)
    plt.close()

In [36]:
plot_rain_histograms(morning_commutes=morning_commutes, 
                                    evening_commutes=evening_commutes,
                                    bins=100, 
                                    y_ticks = [y for y in range(0, 12500, 2500)], 
                                    x_ticks = [x*60 for x in range(1, 16)], 
                                    has_grid = False, 
                                    legend_loc = 1, 
                                    frameon=True,
                                    file_prefix='all_stations',
                                    title_prefix='',
                                    write=DO_WRITE_CHARTS)

../../charts/statistical_analysis/rain_all_stations_commutes.png


# Impact of Rain on Morning Commuter Traffic


<div class="alert alert-info">

<p><b>31. Morning Rainy Commutes</b></p>

<p>A <b>Wilcoxon Rank-Sum Statistic Test</b> is appropriate for this problem as we are trying to see a difference between two sample means from data sets with very different values.</p>

<ul>
    <li>$H$o : Morning Commuter Mean Number of trips on Rainy Days = Morning Commuter Mean Number of trips on Dry Days</li>
    <li>$H$a : Morning Commuter Mean Number of trips on Rainy Days ≠ Morning Commuter Mean Number of trips on Dry Days</li>
</ul>
</div>

In [37]:
morning_rain_commute_t_test =  monthly_tests(df=ss_morning_commutes, test_cat='rain')
morning_rain_commute_t_test

/Users/samgutentag/anaconda/envs/bikeshare/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3146: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/Users/samgutentag/anaconda/envs/bikeshare/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/samgutentag/anaconda/envs/bikeshare/lib/python3.6/site-packages/scipy/stats/stats.py:4956: RuntimeWarning: invalid value encountered in double_scalars
  z = (s - expected) / np.sqrt(n1*n2*(n1+n2+1)/12.0)


,day_ratio,diff_of_means,drop_share,dry_dates,dry_mean,rainy_dates,rainy_mean,sig_diff_of_means,sig_drop_share,t_can_reject,t_p_value,t_score,z_can_reject,z_p_value,z_score
0,0.189655,-32.031348,-28.407006,58,112.758621,11,80.727273,-32.031348,-28.407006,True,3.290381e-02,-2.178340,True,0.011864,-2.516168
1,0.203704,-49.397306,-39.884189,54,123.851852,11,74.454545,-49.397306,-39.884189,True,9.492971e-04,-3.468469,True,0.002696,-3.000472
2,0.169492,-50.866102,-38.544824,59,131.966102,10,81.100000,-50.866102,-38.544824,True,8.443150e-05,-4.186607,True,0.003017,-2.965957
3,0.118644,-59.559322,-41.778623,59,142.559322,7,83.000000,-59.559322,-41.778623,True,4.191349e-07,-5.636741,True,0.000688,-3.394422
4,0.015385,-98.430769,0.000000,65,138.430769,1,40.000000,0.000000,0.000000,False,0.000000e+00,0.000000,False,0.121496,-1.548526
5,0.030769,-89.276923,0.000000,65,146.276923,2,57.000000,0.000000,0.000000,True,3.759187e-05,-4.425275,True,0.020278,-2.321160
6,0.030303,-136.227273,0.000000,66,139.727273,2,3.500000,0.000000,0.000000,True,9.150735e-09,-6.575373,True,0.018307,-2.359351
7,0.000000,-144.750000,0.000000,68,144.750000,0,0.000000,0.000000,0.000000,False,0.000000e+00,0.000000,False,0.000000,0.000000
8,0.062500,-74.156250,-64.818356,64,114.406250,4,40.250000,-74.156250,-64.818356,True,1.378614e-02,-2.530404,True,0.016489,-2.397916
9,0.029412,-66.529412,0.000000,68,128.029412,2,61.500000,0.000000,0.000000,True,2.895403e-02,-2.231412,False,0.094032,-1.674502


In [38]:
plot_pvalue_test(test_results=morning_rain_commute_t_test, a='Rain', b='Normal', 
                 title='morning', write=DO_WRITE_CHARTS, lower_legend_loc=4, upper_legend_loc=4)

../../charts/statistical_analysis/morning_rain_commute_pvalue.png
../../charts/statistical_analysis/morning_rain_commute_diff.png
../../charts/statistical_analysis/morning_rain_commute_drop.png


# Impact of Rain on Evening Commuter Traffic


<div class="alert alert-info">

<p><b>2b. Evening Rainy Commutes</b></p>

<p>A <b>Wilcoxon Rank-Sum Statistic Test</b> is appropriate for this problem as we are trying to see a difference between two sample means from data sets with very different values.</p>

<ul>
    <li>$H$o : Evening Commuter Mean Number of trips on Rainy Days = Evening Commuter Mean Number of trips on Normal Days</li>
    <li>$H$a : Evening Commuter Mean Number of trips on Rainy Days ≠ Evening Commuter Mean Number of trips on Normal Days</li>
</ul>
</div>

In [39]:
evening_rain_commute_t_test =  monthly_tests(df=ss_evening_commutes, test_cat='rain')
evening_rain_commute_t_test

/Users/samgutentag/anaconda/envs/bikeshare/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3146: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/Users/samgutentag/anaconda/envs/bikeshare/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/samgutentag/anaconda/envs/bikeshare/lib/python3.6/site-packages/scipy/stats/stats.py:4956: RuntimeWarning: invalid value encountered in double_scalars
  z = (s - expected) / np.sqrt(n1*n2*(n1+n2+1)/12.0)


,day_ratio,diff_of_means,drop_share,dry_dates,dry_mean,rainy_dates,rainy_mean,sig_diff_of_means,sig_drop_share,t_can_reject,t_p_value,t_score,z_can_reject,z_p_value,z_score
0,0.175439,-31.214035,-26.539379,57,117.614035,10,86.400000,-31.214035,-26.539379,True,3.733365e-02,-2.125688,False,0.050809,-1.953088
1,0.148148,-53.574074,-43.005798,54,124.574074,8,71.000000,-53.574074,-43.005798,True,7.410755e-04,-3.556425,True,0.000942,-3.307189
2,0.233333,-61.478571,-47.273027,60,130.050000,14,68.571429,-61.478571,-47.273027,True,9.839172e-08,-5.925245,True,0.000010,-4.416420
3,0.101695,-46.841808,-33.337354,59,140.508475,6,93.666667,-46.841808,-33.337354,True,6.268037e-05,-4.289767,True,0.003217,-2.946187
4,0.015385,-56.861538,0.000000,65,135.861538,1,79.000000,0.000000,0.000000,False,0.000000e+00,0.000000,False,0.134645,-1.496034
5,0.015385,-7.384615,0.000000,65,148.384615,1,141.000000,0.000000,0.000000,False,0.000000e+00,0.000000,False,0.618006,-0.498678
6,0.000000,-138.552239,0.000000,67,138.552239,0,0.000000,0.000000,0.000000,False,0.000000e+00,0.000000,False,0.000000,0.000000
7,0.000000,-150.089552,0.000000,67,150.089552,0,0.000000,0.000000,0.000000,False,0.000000e+00,0.000000,False,0.000000,0.000000
8,0.015625,37.156250,0.000000,64,117.843750,1,155.000000,0.000000,0.000000,False,0.000000e+00,0.000000,False,0.689340,0.399751
9,0.014706,-39.176471,0.000000,68,129.176471,1,90.000000,0.000000,0.000000,False,0.000000e+00,0.000000,False,0.482096,-0.702935


In [40]:
plot_pvalue_test(test_results=evening_rain_commute_t_test, a='Rain', b='Normal', 
                 title='evening', write=DO_WRITE_CHARTS, lower_legend_loc=4, upper_legend_loc=4)

../../charts/statistical_analysis/evening_rain_commute_pvalue.png
../../charts/statistical_analysis/evening_rain_commute_diff.png
../../charts/statistical_analysis/evening_rain_commute_drop.png
